In [5]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/torchvision-0.2.0-py2.py3-none-any.whl (48 kB)
ERROR: Could not find a version that satisfies the requirement torchaudio (from versions: none)
ERROR: No matching distribution found for torchaudio
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install torchvision

  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/ef/a2/f16cac894c4c71585b3411707502ed8d607945fb4a695857621565bd728d/torchvision-0.16.2-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 5.5 MB/s eta 0:00:0000:0100:01


## Importación librerías necesarias

In [24]:
# Importación de las bibliotecas de PyTorch
import torch
import torchvision
import torchvision.transforms as transforms 
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import os

## Creación y entrenamiento del modelo

Primero se crea una clase para crear una red neuronal de dos capa totalmente conectada, tomando imágenes planas de 28x28 como entrada y proporcionando una salida que se pueda usar para clasificar esas imágenes en 10 categorías (de 0 a 9, siendo estos los números que se quieren predecir).

Es un ejemplo típico de una red neuronal básica utilizada para tareas de clasificación de dígitos en el conjunto de datos MNIST.

Para simplificar en el diagrama de red neuronal, la capa oculta sería fc1 y la capa de salida sería fc2.

In [4]:
# Se crea una clase llamada SimpleNet que va a heredar de pytorch.nn.Module

class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 64) #28*28 es la dimensión de la imagen
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28*28) #para aplanar la imagen
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

A continuación se define una función de entrenamiento. 
La primera línea transforms.Compose es un método que encadena varias transformaciones de imágenes. Aquí, está combinando dos transformaciones:
- *transforms.ToTensor()* convierte imágenes PIL o ndarray de NumPy en tensores de PyTorch.
- *transforms.Normalize((0.5,), (0.5,))* normaliza las imágenes tensoras con media y desviación estándar, que son 0.5 para cada canal. Esto se hace porque el conjunto de datos MNIST es en escala de grises, por lo tanto, tiene un solo canal

En la segunda línea se carga el conjunto de datos de MNIST:
- *root=* especifica el directorio donde se almacenará o se almacena el conjunto de datos
- *train=True* indica que se debe descargar el conjunto de entrenamiento
- *download=True* le dice a la función que descargue los datos si aún no están presentes en el directorio especificado
- *transform=transform* aplica las transformaciones definidas anteriormente a cada imagen

En la tercera línea se carga el conjunto de datos y se proporcionan varias funciones de utilidad, configurando los siguientes parámetros:
- *trainset* es el conjunto de datos del cual se cargarán los datos
- *batch_size=4* especifica que se devolverán cuatro muestras por lote
- *shuffle=True* significa que los datos se barajarán en cada epoch

La cuarta línea:
- *net = SimpleNet()* inicializa una instancia de SimpleNet, que es el modelo de red neuronal que se ha definido antes

Quinta línea:
- *criterion = nn.CrossEntropyLoss()* establece la función de pérdida a la Entropía Cruzada, que se utiliza comúnmente para tareas de clasificación

Sexta línea, se define el optimizador para entrenar la red neuronal, utilizando el SDG como algoritmo de optimización:
- *net.parameters()* proporciona los parámetros del modelo SimpleNet que serán optimizados
- *lr=0.001* establece la tasa de aprendizaje, que determina el tamaño del paso en cada iteración mientras se avanza hacia un mínimo de la función de pérdida
- *momentum=0.9* es un parámetro que ayuda a acelerar el SGD en la dirección relevante y amortigua las oscilaciones

Explicación del bucle:

Cada paso completo a través del conjunto de datos se conoce como un epoch. Durante cada epoch, el modelo intenta aprender y ajustar sus parámetros internos (pesos y sesgos) para poder predecir mejor las etiquetas de los datos de entrada.

Al inicio de cada epoch, se inicializa running_loss a 0.0.

Se itera a través de trainloader, que es un DataLoader de PyTorch. Este DataLoader sirve lotes (batch) de imágenes y etiquetas correspondientes del conjunto de datos MNIST. Cada lote contiene 4 imágenes debido a batch_size=4 especificado al crear trainloader (al principio de la función train()).

*optimizer.zero_grad()* se usa para poner a cero los gradientes de todos los parámetros en el modelo. Esto se hace porque, por defecto, los gradientes se acumulan en PyTorch.

Predicción y Pérdida: Se pasa un lote de imágenes a través de la red (outputs = net(inputs)) y luego se calcula la pérdida utilizando la función de pérdida de entropía cruzada (loss = criterion(outputs, labels)).

Backpropagation: loss.backward() calcula el gradiente de la pérdida con respecto a todos los parámetros del modelo que son diferenciables.

Actualización de Parámetros: optimizer.step() actualiza los parámetros del modelo en dirección que reduce la pérdida, utilizando los gradientes calculados y el algoritmo de optimización (en este caso, SGD con momentum)

In [12]:
# Se define la función de entrenamiento

def train():
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5))])

    trainset = torchvision.datasets.MNIST(root='/Users/n.c.rodriguez/nico/curso_AI/IA generativa/Pytorch', train=True, download=True, transform=transform)

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)

    net = SimpleNet()

    criterion = nn.CrossEntropyLoss()
    
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(2): # loop sobre el dataset 2 veces
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            
            optimizer.zero_grad()
            
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
            if i % 2000 == 1999:    # imprime cada 2000 mini-batches 
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000)) 
                running_loss = 0.0

    # Por último se guarda el modelo en un archivo para poder cargarlo más adelante
    print('Finished Training')
    torch.save(net.state_dict(), 'modelo_mnist.pth')

In [13]:
# Usamos el método __init__ para que cada vez que llamemos a nuestro módulo se cargue la función train por defecto.
if __name__ == "__main__": 
    train()

100%|██████████| 9912422/9912422 [00:01<00:00, 7314636.08it/s]


Extracting /Users/n.c.rodriguez/nico/curso_AI/IA generativa/Pytorch/MNIST/raw/train-images-idx3-ubyte.gz to /Users/n.c.rodriguez/nico/curso_AI/IA generativa/Pytorch/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 1000319.53it/s]


Extracting /Users/n.c.rodriguez/nico/curso_AI/IA generativa/Pytorch/MNIST/raw/train-labels-idx1-ubyte.gz to /Users/n.c.rodriguez/nico/curso_AI/IA generativa/Pytorch/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 6202437.42it/s]


Extracting /Users/n.c.rodriguez/nico/curso_AI/IA generativa/Pytorch/MNIST/raw/t10k-images-idx3-ubyte.gz to /Users/n.c.rodriguez/nico/curso_AI/IA generativa/Pytorch/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 5714015.83it/s]


Extracting /Users/n.c.rodriguez/nico/curso_AI/IA generativa/Pytorch/MNIST/raw/t10k-labels-idx1-ubyte.gz to /Users/n.c.rodriguez/nico/curso_AI/IA generativa/Pytorch/MNIST/raw

[1,  2000] loss: 0.652
[1,  4000] loss: 0.376
[1,  6000] loss: 0.337
[1,  8000] loss: 0.295
[1, 10000] loss: 0.264
[1, 12000] loss: 0.235
[1, 14000] loss: 0.224
[2,  2000] loss: 0.180
[2,  4000] loss: 0.176
[2,  6000] loss: 0.178
[2,  8000] loss: 0.168
[2, 10000] loss: 0.182
[2, 12000] loss: 0.163
[2, 14000] loss: 0.153
Finished Training


## Uso del modelo

In [14]:
# Vamos a usar torchvision para "leer" o predecir los números escritos a mano en el directorio raíz:
#/Users/n.c.rodriguez/nico/curso_AI/IA generativa/Pytorch
# Carga del modelo
model = SimpleNet()
model.load_state_dict(torch.load('modelo_mnist.pth'))
model.eval()  # Importante para decirle al modelo que ahora está en modo de evaluación

SimpleNet(
  (fc1): Linear(in_features=784, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)

In [29]:
images = []
for root, dirs, files in os.walk('/Users/n.c.rodriguez/nico/curso_AI/IA generativa/Pytorch'):
    for file in files:
        base, extension = os.path.splitext(file) 
        if (extension.lower() == '.jpg'):
            full_path = os.path.join(root, file) 
            images.append(full_path)

Esta es la función de predicción, admite una imagen y la pasa a través del modelo para interpretar el 
número escrito a mano. Primero se aplican las tranformaciones para estadarizar la imagen a predecir:

In [26]:
def predict(image_path):
    # Se carga la imagen desde la ruta
    image = Image.open(image_path)

    # Se extrae el nombre del archivo de la ruta
    _, filename = os.path.split(image_path)

    # Se definen las transformaciones que se deben aplicar a la imagen para que coincida con el formato MNIST 
    transformations = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((28, 28)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    
    # Se aplican las transformaciones a la imagen
    img_transformed = transformations(image)
    
    # Se convierte la imagen para que tenga una dimensión de lote (que PyTorch espera) 
    img_batch = img_transformed.unsqueeze(0)  # Agrega un batch dimension en la posición 0
    
    # Hacer la predicción
    with torch.no_grad():  # No es necesario seguir el rastro de los gradientes
        outputs = model(img_batch)
        _, predicted = torch.max(outputs, 1)

    # El resultado 'predicted' es el índice del dígito que el modelo predice
    print(f"El modelo predice que el dígito del archivo {filename} es: {predicted.item()}")

In [30]:
# Predicción
for image_path in images:
    predict(image_path)

El modelo predice que el dígito del archivo numero8.jpg es: 7
El modelo predice que el dígito del archivo numero10.jpg es: 7
El modelo predice que el dígito del archivo numero1.jpg es: 7
El modelo predice que el dígito del archivo numero2.jpg es: 6
El modelo predice que el dígito del archivo numero4.jpg es: 1
El modelo predice que el dígito del archivo numero5.jpg es: 7
